# パラメタと状態の推定: ベイズ推論とHMC法

- ハミルトニアンモンテカルロ法(Hamiltonian Monte Carlo, HMC法): 乱数生成のためのアルゴリズム

パラメタ推定と乱数生成の繋がりを理解していくことが大切。

## パラメタ推定と乱数生成アルゴリズムの関係

状態空間モデルのパラメータをベイズ推論で求めることが出来そうだが、確率密度関数が複雑で積分ができない。  
そこでパラメタの「事後分布の確率密度関数」に従う乱数を発生させることで、**事後期待値(EAP)** 推定量を割り出すことができる。  
この乱数を発生させる手法がHMC法で、StanがHMC法を行う。

In [1]:
# install.packages('rstan', repos='https://cloud.r-project.org/', dependencies=TRUE)

In [2]:
pkgbuild::has_build_tools(debug = TRUE)

[1] TRUE

In [3]:
Sys.which("make")

make 
"C:\\rtools40\\usr\\bin\\make.exe"

In [4]:
library(rstan)
library(ggplot2)

# rstan設定
rstan_options(auto_write = TRUE)
options(ms.cores = parallel::detectCores())

Loading required package: StanHeaders

Loading required package: ggplot2

rstan (Version 2.21.1, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)

For improved execution time, we recommend calling
Sys.setenv(LOCAL_CPPFLAGS = '-march=native -mtune=native')



In [5]:
# 図示設定
library(repr)
# グラフのオプション
options(repr.plot.width=14, repr.plot.height=6)

## シミュレーションデータの作成

In [6]:
# data
n_sample <- 100 # サンプルサイズ
y <- numeric(n_sample)  # 観測値

# parameters

# 状態の初期値
mu_zero <- 100

# 状態の推定値
mu <- numeric(n_sample)

# 過程誤差の分散
s_w <- 1000

# 観測誤差の分散
s_v <- 5000

In [7]:
# 分析対象となるデータをシミュレーションで作成する
set.seed(1)

# 状態の初期値から最初の時点の状態が得られる
mu[1] <- rnorm(n=1, mean = mu_zero, sd = sqrt(s_w))

# 状態方程式に伴い、状態が繊維する
for(i in 2:n_sample){
    mu[i] <- rnorm(n=1, mean=mu[i-1], sd=sqrt(s_w))
}

# 観測方程式に従い、観測地が得られる
for(i in 1:n_sample){
    y[i] <- rnorm(n=1, mean=mu[i], sd=sqrt(s_v))
}

- 1時点目の状態muは、状態の初期値mu_zeroに過程誤差が加わったもの
- 2時点目以降の状態は、前時点の状態に過程誤差が加わったもの
- 観測地は、同じ時点における状態に、観測誤差が加わったもの

## Stanファイルの記述

Stanは確立的プログラミング言語と呼ばれるものの一種。  
- 「どの変数が、どのような確率分布に従って生成されているか」をそのまま記述して、モデルを構築することができる。  
- データを確率変数としてみなし、確率変数の従う確率分布を推定したいとき、コードとして書くことができる。

In [9]:
# データの準備
data_sim <- list(y = y, n_sample = n_sample)

# モデルの推定
fit_stan_1 <- stan(
    file = "6-3-local-level-model.stan",
    data = data_sim,
    iter = 550,
    warmup = 50,
    thin = 1,
    chains = 4,
    seed = 1
)

ERROR: Error in compileCode(f, code, language = language, verbose = verbose): Compilation ERROR, function(s)/method(s) not created! C:/rtools40/mingw64/bin/../lib/gcc/x86_64-w64-mingw32/8.3.0/../../../../x86_64-w64-mingw32/bin/ld.exe: file22787564e83.o:file22787564e83.cpp:(.text+0x6bc): undefined reference to `tbb::internal::task_scheduler_observer_v3::observe(bool)'
C:/rtools40/mingw64/bin/../lib/gcc/x86_64-w64-mingw32/8.3.0/../../../../x86_64-w64-mingw32/bin/ld.exe: file22787564e83.o:file22787564e83.cpp:(.text+0x11f7): undefined reference to `rstan::stan_fit::stan_fit(SEXPREC*, int)'
C:/rtools40/mingw64/bin/../lib/gcc/x86_64-w64-mingw32/8.3.0/../../../../x86_64-w64-mingw32/bin/ld.exe: file22787564e83.o:file22787564e83.cpp:(.text+0x6fe): undefined reference to `tbb::internal::task_scheduler_observer_v3::observe(bool)'
C:/rtools40/mingw64/bin/../lib/gcc/x86_64-w64-mingw32/8.3.0/../../../../x86_64-w64-mingw32/bin/ld.exe: file22787564e83.o:file22787564e83.cpp:(.text$_ZN3tbb8internal26tas

ERROR: Error in sink(type = "output"):  コネクションが不正です 


In [ ]:
library(rstan)
example(stan_model, run.dontrun = TRUE)

In [ ]:
# 出力の結果と収束の判定
# 得られた乱数の要約をみた

options(max.print=10000)
print(fit_stan_1)

In [ ]:
print(fit_planer)

In [ ]:
print(
    fit_stan_1, # 推定結果
    digits = 1, # 小数点桁数
    pars = c("s_w", "s_w", "lp__"),  # 表示するパラメタ
    probs = c(0.025, 0.5, 0.975)  # 区間幅の設定
)

In [ ]:
traceplot(fit_stan_1, pars = c("s_w", "s_v"))